In [5]:
import pandas as pd
import numpy as np

# === 1) Load your Excel ===
path = r"C:/Users/User/Downloads/통합 문서1.xlsx"   # <-- update filename
df = pd.read_excel(path, sheet_name=0)

# === 2) Ensure numeric conversion ===
df_num = df.copy()
for col in df_num.columns:
    df_num[col] = pd.to_numeric(df_num[col], errors="coerce")

# === 3) IQR-based outlier detection ===
outlier_union = pd.Series(False, index=df_num.index)
records = []

for col in df_num.columns:
    s = df_num[col].dropna()
    if len(s) < 4:
        continue  # skip small columns
    
    q1 = np.percentile(s, 25)
    q3 = np.percentile(s, 75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    mask = (df_num[col] < lower) | (df_num[col] > upper)
    outlier_union |= mask

    for idx in df_num.index[mask]:
        records.append({
            "컬럼": col,
            "행": idx,
            "값": df_num.loc[idx, col],
            "Q1": q1, "Q3": q3, "IQR": iqr,
            "하한": lower, "상한": upper
        })

# === 4) Results ===
outlier_report = pd.DataFrame(records).sort_values(["컬럼","행"]).reset_index(drop=True)
outliers_any = df.loc[outlier_union]

print("총 아웃라이어 행 수:", len(outliers_any))
print("\n=== 아웃라이어 요약 ===")
print(outlier_report)

# Optionally save to Excel
# outlier_report.to_excel("outlier_report.xlsx", index=False)
# outliers_any.to_excel("rows_with_any_outlier.xlsx", index=False)


총 아웃라이어 행 수: 32

=== 아웃라이어 요약 ===
             컬럼   행     값    Q1    Q3   IQR    하한    상한
0   Unnamed: 10   9  37.0  36.0  36.0   0.0  36.0  36.0
1   Unnamed: 10  17  35.0  36.0  36.0   0.0  36.0  36.0
2   Unnamed: 10  38  35.0  36.0  36.0   0.0  36.0  36.0
3   Unnamed: 10  39  35.0  36.0  36.0   0.0  36.0  36.0
4   Unnamed: 10  43  35.0  36.0  36.0   0.0  36.0  36.0
5   Unnamed: 10  49  35.0  36.0  36.0   0.0  36.0  36.0
6   Unnamed: 10  56  35.0  36.0  36.0   0.0  36.0  36.0
7   Unnamed: 10  58  35.0  36.0  36.0   0.0  36.0  36.0
8   Unnamed: 10  63  37.0  36.0  36.0   0.0  36.0  36.0
9   Unnamed: 10  68  37.0  36.0  36.0   0.0  36.0  36.0
10  Unnamed: 10  69  35.0  36.0  36.0   0.0  36.0  36.0
11  Unnamed: 11  17  95.0  50.0  65.0  15.0  27.5  87.5
12  Unnamed: 12  17  90.0  60.0  70.0  10.0  45.0  85.0
13  Unnamed: 12  20  90.0  60.0  70.0  10.0  45.0  85.0
14  Unnamed: 12  47  40.0  60.0  70.0  10.0  45.0  85.0
15  Unnamed: 12  48  35.0  60.0  70.0  10.0  45.0  85.0
16   Unnamed: 